In [2]:
import torch
import json
from PIL import Image
import requests
from tqdm import tqdm
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
import sys

In [ ]:
def ansType(ans):
    if ans == 'yes-no':
        return ['yes', 'no']
    elif is_number(ans):
        return 'number'
    elif ans in ["cube", "sphere", "cylinder"] :
        return 'shape'
    elif ans in  ["small", "large"]:
        return 'size'
    elif ans in [ "left", "right", "behind", "front"]:
        return 'relation'
    elif ans in [ "gray", "red", "blue", "green", "brown", "purple", "cyan", "yellow"]:
        return 'colour'
    elif ans in ["rubber", "metal"]:
        return 'material'
    
    return 'none'

In [22]:
# image_path = '/scratch/nmachav1/CLEVR_v1.0/images/val/'
# model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b")
# processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")

# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

image_path = '/scratch/nmachav1/CLEVR_v1.0/images/val/'
#object_type = ['inter', 'intra']
for i in range(3,4):
    dataset = f'/scratch/nmachav1/MLLM_Hallucinations_CLEVR/dataset/object_number_split/val_num_{i}.json'
    #output_path = f'/scratch/nmachav1/MLLM_Hallucinations_CLEVR/outputs/InstructBLIP/val_num_{i}.json'
    #dataset = f'/scratch/nmachav1/MLLM_Hallucinations_CLEVR/dataset/object_type_split/val_type_{i}.json'
    #output_path = f'/scratch/nmachav1/MLLM_Hallucinations_CLEVR/outputs/LLaVA/val_type_{i}.json'
    with open(dataset, 'r') as f:
        data = json.load(f)


    complete_data = []
    for each in tqdm(data[6:7]):
        each_copy = each.copy()
        img = Image.open(image_path+each['image_filename']).convert("RGB")
        prompt = each['question'] + 'Choose one of the following: gray, blue, brown, cyan, yellow, purple, red.'
        inputs = processor(images=img, text=prompt, return_tensors="pt").to(device)
        outputs = model.generate(
            **inputs,
            do_sample=False,
            use_cache=True,
            num_beams=5,
            max_length=256,
            min_length=1,
            top_p=1.0,
            repetition_penalty=1.0,
            length_penalty=1,
            top_k=50,
            temperature=1e-05
        )
        generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
        each_copy['model_generated_answer'] = generated_text
        print(each_copy)
        #complete_data.append(each_copy)


    # with open(output_path, 'w') as f:
    #     json.dump(complete_data, f ,indent=4)




100%|██████████| 1/1 [00:00<00:00,  1.42it/s]

{'image_filename': 'CLEVR_val_000012.png', 'question_index': 121, 'answer_type': 'colour', 'question': 'What is the color of the rubber cube?', 'ground_truth': 'red', 'model_generated_answer': 'red', 'consistent': ''}
